In [1]:
pip install vaderSentiment

In [2]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

ModuleNotFoundError: No module named 'textblob'

In [ ]:
from google.colab import files
files.upload()

In [ ]:
#load the data
df1 = pd.read_csv("News.csv")
df2 = pd.read_csv("Stock.csv")

In [ ]:
df1.head()

In [ ]:
df1.shape


In [ ]:
df2.tail()

In [ ]:
df2.shape

In [ ]:
#creating our datasets by merging these 2 cells based on the dates
merge = df1.merge(df2, how='inner', on='Date', left_index=True)

In [ ]:
merge.head()

In [ ]:
#combine the top news headline into a coloumn
headlines = []
for row in range(len(merge.index)):
    headlines.append(" ".join(str(x) for x in merge.iloc[row,2:27])) 

In [ ]:
headlines[0]

In [ ]:
#clean the data
clean_headlines = []
for i in range(0,len(headlines)):
    clean_headlines.append(re.sub("b[(')]", '',headlines[i]))
    clean_headlines[i] = re.sub('b[(")]', '', clean_headlines[i]) #remove b"
    clean_headlines[i] = re.sub("\'", '', clean_headlines[i])    #remove \'
    

In [ ]:
clean_headlines[0]

In [ ]:
#adding clean headline to merge dataset
merge['Combined_News'] = clean_headlines

In [ ]:
merge['Combined_News'][0]

In [ ]:
merge.head()

In [ ]:
#create function to get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:

merge["Subjectivity"] = merge['Combined_News'].apply(getSubjectivity)
merge["Polarity"] = merge['Combined_News'].apply(getPolarity)

In [ ]:
#Show the new columns
merge.head()

In [ ]:
#create a function to get sentiment scores
def getSIA(text):
  sia = SentimentIntensityAnalyzer()
  sentiment = sia.polarity_scores(text)
  return sentiment

In [ ]:
#get the sentment scores for each day
compound = []
neg = []
pos = []
neu = []
SIA = 0
for i in range(0,len(merge['Combined_News'])):
  SIA = getSIA(merge['Combined_News'][i])
  compound.append(SIA['compound'])
  neg.append(SIA['neg'])
  neu.append(SIA['neu'])
  pos.append(SIA['pos'])

In [ ]:
#store our values in the merge data set
merge['Compound'] = compound
merge['Negative'] = neg
merge['Positive'] = pos
merge['Neutral'] = neu

In [ ]:
merge.head()

In [ ]:
#create a list of columns i want to keep
keep_columns = ['Open','High','Low','Volume','Subjectivity','Polarity','Compound','Negative','Neutral','Positive','Label']
df = merge[keep_columns]
df

In [ ]:
#create the features dataset
X = np.array(df.drop(["Label"],1))
#create the target dataset
y = np.array(df['Label'])

In [ ]:
#test train split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
model = LinearDiscriminantAnalysis()
model.fit(x_train,y_train)

In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
y_test

In [ ]:
# show the model metrics
print(classification_report(y_test, predictions))